In [1]:
import pandas as pd
import numpy as np
import itertools
import datetime
import pandas_gbq
import matplotlib.pyplot as plt
from datetime import *
from datetime import datetime, timedelta, date
# %load_ext google.colab.data_table
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
project_id = "perceptive-ivy-290216"

# Standard plotly imports
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
query2=f"""
SELECT * FROM `perceptive-ivy-290216.f1_api.qualifying_lap_time`
# WHERE YEAR=2023
# AND GP='Spanish Grand Prix'
"""
track3=pandas_gbq.read_gbq(query2,project_id,dialect='standard')

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/google/cloud/bigquery/table.py:2379: UserWarning: A progress bar was requested, but there was an error loading the tqdm library. Please install tqdm to use the progress bar functionality.
  record_batch = self.to_arrow(


In [4]:
track2=track3[(track3["GP"]=='Belgian Grand Prix')&(track3["Year"]==2024)]
year=track2['Year'].iloc[0]
gp=track2['GP'].iloc[0]

In [5]:
Drivers=(track2['Driver'].unique())
Drivers

array(['VER', 'LEC', 'PER', 'HAM', 'NOR', 'PIA', 'RUS', 'SAI', 'ALO',
       'OCO', 'ALB', 'GAS', 'RIC', 'BOT', 'STR', 'HUL', 'MAG', 'TSU',
       'SAR', 'ZHO'], dtype=object)

In [6]:
track2["LapTime"].min()

'0 days 00:01:53.159000'

In [7]:
#Assign Rank for each entry point
track2["RK"] = track2.groupby(by=["Driver"])["LapTime"].rank(method="dense", ascending=True)
track2["RK"]= track2["RK"].astype(int)

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_2894/2442521796.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  track2["RK"] = track2.groupby(by=["Driver"])["LapTime"].rank(method="dense", ascending=True)
/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_2894/2442521796.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  track2["RK"]= track2["RK"].astype(int)


In [8]:
track_fastest=track2[track2['RK']==1]

In [9]:
pole_lap = track_fastest["LapTime"].min()
track_fastest['LapTimeDelta']= pd.to_timedelta(track_fastest["LapTime"]) - pd.to_timedelta(pole_lap)
track_fastest['LapTimeDelta']=track_fastest['LapTimeDelta'].astype(str)

for index, row in track_fastest.iterrows():
  if track_fastest.loc[index, 'LapTimeDelta']=='0 days 00:00:00':
    track_fastest.loc[index, 'LapTimeDelta']='0 days 00:00:00.01'
track_fastest['LapTimeDelta']=pd.to_timedelta(track_fastest['LapTimeDelta'])
track_fastest

track_fastest['LapTimeDelta'] = track_fastest['LapTimeDelta'] + pd.to_datetime('1970/01/01')

track_fastest['LapTimeDelta2']= pd.to_timedelta(track_fastest["LapTime"]) - pd.to_timedelta(pole_lap)
track_fastest['LapTimeDelta2']=track_fastest['LapTimeDelta2'].astype(str)

track_fastest["LapTimeDelta2"]=track_fastest['LapTimeDelta2'].str.split('days ').str[1]

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_2894/2506531996.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  track_fastest['LapTimeDelta']= pd.to_timedelta(track_fastest["LapTime"]) - pd.to_timedelta(pole_lap)
/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_2894/2506531996.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  track_fastest['LapTimeDelta']=track_fastest['LapTimeDelta'].astype(str)
/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_2894/2506531996.py:8: Sett

In [10]:
fig=px.bar(
    track_fastest,
    y="Driver",
    x='LapTimeDelta',
    text='LapTimeDelta2',
    orientation='h',
    color='Driver',
    template="presentation",
    hover_data=['Team', 'Year', 'GP','LapTime', 'LapNumber','Sector1Time', 'Sector2Time', 'Sector3Time',
       'Compound', 'TyreLife', 'FreshTyre'],
    title="<b>Qualifying Delta vs. Fastest Lap for the {} {}</b>".format(year,gp),
    height=800, 
    width=1200,

color_discrete_map={
                 "ALB": "#64C4FF",
                 "ALO": "#229971",
                 "BOT": "#52e252",
                 "GAS": "#0093cc",
                 "HAM": "#27F4D2",
                 "HUL": "#B6BABD",
                 "LEC": "#E80020",
                 "MAG": "#B6BABD",
                 "NOR": "#FF8000",
                 "OCO": "#0093cc",
                 "PER": "#3671C6",
                 "PIA": "#FF8000",
                 "RIC": "#6692FF",
                 "RUS": "#27F4D2",
                 "SAI": "#E80020",
                 "SAR": "#37BEDD",
                 "STR": "#229971",
                 "TSU": "#6692FF",
                 "VER": "#3671C6",
                 "ZHO": "#52e252"
                 }
)
fig.update_layout(
    yaxis = dict(tickfont = dict(size=20)),
    xaxis = dict(tickfont = dict(size=20)),

    xaxis_title="<b>Delta</b>",
    yaxis_title="<b>Driver</b>",
    title_font_family="<b>PT Sans Narrow</b>",

)

fig.update_layout(xaxis_tickformat='%H:%M:%S.%f')

fig.update_traces(marker_line_width=1,marker_line_color="BLACK")

fig.update_layout(yaxis={'categoryorder':'total descending'})

fig.update_traces(textposition='auto')

fig.update_layout(
    title_x=0.5,
    hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="PT Sans Narrow"
    ),
    yaxis = dict(tickfont = dict(size=20)),
    xaxis = dict(tickfont = dict(size=20)),
    font=dict(
        family="PT Sans Narrow",
        size=14,
        color="Black"
    ),
    title_font_family="PT Sans Narrow",
    margin=dict(l=60, r=5, t=35, b=60),
)
fig.show()

In [11]:
fig.write_html("{} {} Qualifying Delta.html".format(year,gp))